In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [27]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.tail()

,Model_name,input_shape,memory,input_width,input_size,nb_layers,sum_activations,params
153,basic_model_448_37,"(1, 448, 448, 3)",2427.5,448,602112,62,190772012,9215748
154,basic_model_112_11,"(1, 112, 112, 3)",2124.0,112,37632,22,5828752,4989982
155,basic_model_480_9,"(1, 480, 480, 3)",2033.0,480,691200,19,34142880,2556232
156,basic_model_320_15,"(1, 320, 320, 3)",1886.0,320,307200,30,23250476,1610240
157,basic_model_90_4,"(1, 90, 90, 3)",1796.5,90,24300,9,6439860,453340


In [49]:
# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers',
                'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.tail()

,Model_name,input_shape,memory,input_width,input_size,nb_layers,sum_activations,params
85,MNASNet1.0,"(512, 512, 3)",2440.0,512,786432,153,85346408,4251760
86,MNASNet1.0,"(720, 720, 3)",2210.0,720,1555200,153,169583288,4251760
87,EfficientNet-B4,"(90, 90, 3)",3061.0,90,24300,470,10284484,19466816
88,EfficientNet-B4,"(512, 512, 3)",3278.0,512,786432,470,305708296,19466816
89,EfficientNet-B4,"(720, 720, 3)",3604.0,720,1555200,470,607426440,19466816


In [29]:
train_dataset.pop('input_width')

0       448
1      1792
2       299
3      1792
4       299
       ... 
153     448
154     112
155     480
156     320
157      90
Name: input_width, Length: 158, dtype: int64

In [50]:
test_dataset.pop('input_width')

0      90
1     512
2     720
3      90
4     512
     ... 
85    512
86    720
87     90
88    512
89    720
Name: input_width, Length: 90, dtype: int64

In [31]:
Y = train_dataset.pop('memory')
X = train_dataset
Y_f = Y
X_f = X

In [32]:
X.tail()

,Model_name,input_shape,input_size,nb_layers,sum_activations,params
153,basic_model_448_37,"(1, 448, 448, 3)",602112,62,190772012,9215748
154,basic_model_112_11,"(1, 112, 112, 3)",37632,22,5828752,4989982
155,basic_model_480_9,"(1, 480, 480, 3)",691200,19,34142880,2556232
156,basic_model_320_15,"(1, 320, 320, 3)",307200,30,23250476,1610240
157,basic_model_90_4,"(1, 90, 90, 3)",24300,9,6439860,453340


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [34]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [35]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [36]:
X_f_model_names = X_f.pop('Model_name')
X_f_input_shapes = X_f.pop('input_shape')
dtrain_f = xgb.DMatrix(X_f, label=Y_f)

In [37]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [51]:
Y_e = test_dataset.pop('memory')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [39]:
params = {
    # Parameters that we are going to tune.
    'max_depth':16,
    'min_child_weight': 5,
    'eta':.05,
    'subsample': .8,
    'colsample_bytree': 1,
    'gamma' : 0.5,
    # Other parameters
    'objective':'reg:squarederror',
}

In [40]:
params['eval_metric'] = "rmse"
num_boost_round = 3000

In [41]:
print(params)

{'max_depth': 16, 'min_child_weight': 5, 'eta': 0.05, 'subsample': 0.8, 'colsample_bytree': 1, 'gamma': 0.5, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}


In [42]:
num_boost_round = 93
best_model = xgb.train( 
    params,
    dtrain_f,
    num_boost_round=num_boost_round,
)

In [43]:
#best_model.save_model("model_16-04.model")
loaded_model = best_model

In [52]:
y_pred = loaded_model.predict(dtest_e)
y_test = Y_e.values

In [53]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9851127105578017
R2_value :  0.984886433092004
Mean absolute error MAE : 125.20166422526042
Root Mean squared error RMSE : 157.31290220913914
Median absolute error MAD: 104.474609375


In [54]:
# Calculate mean absolute percentage error (MAPE)
errors = abs(y_pred-y_test)
mape = 100 * (errors / y_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE :', max(mape))
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE : 20.673723118279568
MAPE: 4.1 %.
Accuracy: 95.9 %.


In [55]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [56]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, y_test[i], y_pred[i],abs(y_pred[i]-y_test[i]), ((abs(y_pred[i]-y_test[i]))/y_test[i])*100])